#### Funções

In [1]:
import pandas as pd
from collections import Counter
import random
from utils.Gpt_generator import Gpt_generator
from utils.Json_controller import Json_controller
import time

In [2]:
def get_data(dir= 'data', split= 'train', track= 'a', language= 'ptbr'):
    
    archive = language + '.csv' if split == 'train' else language + '_' + track + '.csv'

    path = f'{dir}/{split}/track_{track}/{archive}'
    
    return pd.read_csv(path)

## Abordagem 1

Vamos contemplar, primeiramente, a geração de dados sintéticos para a track A, onde temos uma classificação binária de emoções.

Seguiremos a abordagem do artigo: https://aclanthology.org/2024.acl-long.120.pdf


Focaremos na variabilidade desses dados, e uma abordagem few shot, iremos variar na seguinte medida:


    - Emoções (faremos isso dinâmico em função do dataset (a probabilidade será proporcional a quantidade de amostras))
    - Tamanho do tweet (amostrados em função dos tamanhos reais)
    - Intensidade do sentimento (leve, moderado, forte)



Os dados serão retornados em formato de JSON

In [3]:
df = get_data()

'''Vamos estratificar o dataset, tendo somente uma coluna de emoções, isso nos ajuda, pois teremos agora também as 
combinações de emoções, posteriormente, vamos amostrar essas emoções para geração de novos dados, dando maior probabilidade a 
emoções ou combinações mais raras, atualizando isso de maneira dinâmica para atualizar a geração.'''


emotion_columns = ['Anger', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise']
df['labels'] = df[emotion_columns].apply(
    lambda row: ' '.join([col for col, val in row.items() if val == 1]),
    axis=1
)


df['labels'] = df['labels'].apply(
    lambda x: 'Neutral' if x == '' else x
)

df = df.drop(columns=emotion_columns)

In [4]:
def categorize(length):
        if length <= 11:
            return "Pequeno"
        elif length <= 26:
            return "Médio"
        elif length >= 26:
            return "Grande"
        

'''Gera a distribuição de tamanhos de texto'''
category_counts = dict(Counter(df['text'].apply(lambda x: categorize(len(x.split())))))
'''Gera a distribuição de emoções'''
emotion_counts = dict(Counter(df['labels']))

'''Gera uma distribuição da intensidade de emoções (baseados nos dados da track b)'''
aux = get_data(track='b')
intensity_counts = {
    emotion: (aux[aux[emotion] > 0][emotion].value_counts(normalize=True) * 100).round(2).to_dict()
    for emotion in emotion_columns
}

intensity_counts = {
    emotion: {
        "baixo": values.get(1, 0), 
        "moderado": values.get(2, 0),
        "alto": values.get(3, 0)
    }
    for emotion, values in intensity_counts.items()
}

In [5]:
def amostra_categoria_e_emocao(label_dict, category_dict, intensity_counts):
    '''Amostraremos baseado no tamanhjo do texto, emoção e intensidade da emoção.
    
    Para o tamanho e emoção, temos um dicionario dinâmico, onde as probabilidades da amostragem de cada emoção mudam ao longo
    do tempo, baseado na quantidade de vezes que cada emoção foi amostrada. Para a intensidade, faremos de maneira estática, visto que
    dados menos comuns são dados extremos (emoção alta), o que é muito fácil de se identificar no caso da track A. Por isso, usaremos como
    probabilidade a presença de cada intensidade em cada emoção nos dados da track b, adicionamos essa "feature" apenas para gerar mais 
    variabilidade.
    '''
    inverse_weights_labels = {label: 1 / count for label, count in label_dict.items() if count > 0}
    total_weight_labels = sum(inverse_weights_labels.values())
    probabilities_labels = {label: weight / total_weight_labels for label, weight in inverse_weights_labels.items()}
    labels = list(probabilities_labels.keys())
    weights_labels = list(probabilities_labels.values())
    sampled_label = random.choices(labels, weights=weights_labels, k=1)[0]
    label_dict[sampled_label] += 1

    inverse_weights_categories = {category: 1 / count for category, count in category_dict.items() if count > 0}
    total_weight_categories = sum(inverse_weights_categories.values())
    probabilities_categories = {category: weight / total_weight_categories for category, weight in inverse_weights_categories.items()}
    categories = list(probabilities_categories.keys())
    weights_categories = list(probabilities_categories.values())
    sampled_category = random.choices(categories, weights=weights_categories, k=1)[0]
    category_dict[sampled_category] += 1
            
    intensity_emotion = {}
    for emotion in intensity_counts:
        if str(emotion) in sampled_label:
            probabilities = intensity_counts[str(emotion)]
            categories = list(probabilities.keys())
            weights = list(probabilities.values())
            intensity = random.choices(categories, weights=weights, k=1)[0]
            intensity_emotion[emotion] = intensity

    # Caso nenhuma intensidade seja encontrada, o dicionário ficará vazio
    if not intensity_emotion:
        intensity_emotion = {"Nenhuma": "intensidade encontrada"}

    return intensity_emotion, sampled_category

### Geração do prompt

In [6]:
PROMPT = """
Você é um assistente criativo especializado em criar textos sintéticos que simulam postagens informais, mas realistas,
para redes sociais. Esses textos devem refletir as emoçõessolicitadas de maneira genuína e natural, sem exageros ou estereótipos. 

Para criar esses textos:
- Use gírias, abreviações e expressões informais comuns no ambiente online caso faça sentido.
- Os textos podem, mas não precisam, conter xingamentos ou linguagem inapropriada.
- Não use emojis ou hashtags, a menos que sejam essenciais para a naturalidade do texto.
- Insira as emoções indicadas com as intensidades especificadas de maneira criativa, mas coerente com o texto. 
Não é necessário usar as mesmas palavras das emoções, mas elas devem ser perceptíveis no contexto.
- Atenda ao tamanho do texto solicitado.
- o texto deve conter apenas as emoções solicitadas

Exemplos:
Os exemplos abaixo mostram o estilo esperado, não necessariamente as emoções esperadas.
Leia com atenção para entender como transmitir as emoções de forma autêntica:

{exemplos}

Sua Tarefa:
- Gerar um texto em português seguindo o estilo, e não as emoções dos exemplos fornecidos.
- Atender às seguintes características:
  - Emoções PARA GERAR (MUITO IMPORTANTE, GERE SOMENTE AS EMOÇÕES SOLICITADAS): {emoções}
  - Tamanho do texto: {tamanho}
- Certifique-se de que as emoções aparecem no texto nas intensidades indicadas, mas sem tornar isso explícito ou forçado.

Regras Importantes:
1. Não copie os exemplos diretamente. Use sua criatividade para criar algo original.
2. O texto deve parecer uma postagem real, como se fosse escrito por uma pessoa comum.
3. Evite clichês ou exageros que possam tornar o texto artificial.
4. Foque na linguagem e tom apropriados para um post informal, mas autêntico.
5. Se a emoção for Neutral, gere um texto sem emoções, algo como uma notícia ou fato do dia a dia, ou qualquer coisa que não expresse emoções.

Gere o texto solicitado abaixo:
"""


def sample_data_for_emotions(df, emotion_intensity):
    required_emotions = set(emotion_intensity.keys())
    def contains_all_emotions(label):
        label_emotions = set(label.split())
        return required_emotions.issubset(label_emotions)
    filtered_df = df[df['labels'].apply(contains_all_emotions)]

    return filtered_df


def few_shot_examples(df, emotion_intensity, n=6, j=2):
    filtered_df = sample_data_for_emotions(df, emotion_intensity)

    if filtered_df.empty:
        return "Nenhum exemplo encontrado com as emoções especificadas."

    examples = filtered_df.sample(min(n-j, len(filtered_df)), random_state=42)
    random_examples = df.sample(j, random_state=42)
    combined_examples = pd.concat([examples, random_examples]).sample(frac=1, random_state=42)

    formatted_examples = [
        f"- Emoções: {row['labels']}\n- Texto: {row['text']}"
        for _, row in combined_examples.iterrows()
    ]

    return '\n\n'.join(formatted_examples)


def get_prompt(intensity_emotion : dict, lenght : str, df : pd.DataFrame):
    #print(intensity_emotion)
    examples = few_shot_examples(df, intensity_emotion)
    emocoes = ', '.join([f"{emocao} com intensidade {intensidade}" for emocao, intensidade in intensity_emotion.items()])
    return PROMPT.format(exemplos=examples, tamanho=lenght, emoções=emocoes)
    

### Geração dos dados

In [7]:
from utils.Gpt_generator import Gpt_generator
llm = Gpt_generator()

#### GERANDO E SALVANDO

In [8]:
json_file = 'balancing_sample_1.json'
json_controller = Json_controller(json_file)
data = json_controller.load_data()

In [9]:
n_examples = 3000

In [10]:
def get_emotion_intensity(n):
    emotions = ['Anger', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise']

    total_per_emotion = 500

    def get_emotion_by_index(index):
        emotion_index = index // total_per_emotion
        return emotions[emotion_index % len(emotions)]

    emotion = get_emotion_by_index(n)
    intensity = "baixo" if random.random() < 0.9 else "moderado"
    
    return {emotion: intensity}

In [11]:
df['labels'].iloc[0].split()

['Sadness']

In [12]:
def get_emotion_intensity_by_index(n):
    while n >= len(df):
        n -= len(df)
        
    emotions = df['labels'].iloc[n].split()
    intensitys = []
    
    intensitys = ["baixo" if random.random() < 0.9 else "moderado" for _ in range(len(emotions))]
    
    emotion_intensity_map = {
        emotion: ('TEXTO SEM EMOÇÃO' if emotion == "Neutral" else intensity)
        for emotion, intensity in zip(emotions, intensitys)
    }
    
    return emotion_intensity_map

In [13]:
df

,id,text,labels
0,ptbr_train_track_a_00001,"minha vó me disse que era frango e eu comi, ti...",Sadness
1,ptbr_train_track_a_00002,Está e a nossa deputada Benedita linda guerrei...,Joy
2,ptbr_train_track_a_00003,só falta as roupas kkkkkkkkkkk,Joy
3,ptbr_train_track_a_00004,Eu tmb. Comecei a sair de casa agora (fui pela...,Sadness
4,ptbr_train_track_a_00005,Peço a Deus que nossos dirigentes tenham realm...,Neutral
...,...,...,...
2221,ptbr_train_track_a_02222,Eu acho que o CAP vai surpreender hein.,Surprise
2222,ptbr_train_track_a_02223,23:59 - Lula sabia de toda a corrupção no seu ...,Anger
2223,ptbr_train_track_a_02224,O Brasil precisa URGENTE de pessoas sérias e c...,Anger
2224,ptbr_train_track_a_02225,Sera que só eu acho que ta passando da hora de...,Anger


In [17]:
i = len(data)
for n in range(i, n_examples):    
    intensity_emotion , sampled_category = amostra_categoria_e_emocao(emotion_counts, category_counts, intensity_counts) 
    
    #intensity_emotion = get_emotion_intensity_by_index(i)
    
    print(f"Generating example {n+1}/{n_examples}   -  EMOTION: {intensity_emotion}    -   CATEGORY: {sampled_category}")
    
    prompt = get_prompt(intensity_emotion, sampled_category, df)
    text = llm.invoke(prompt)
    
    example_id = f'synthetic_{n}'
    
    emotions = { "Anger": 0, "Disgust": 0, "Fear": 0, "Joy": 0, "Sadness": 0, "Surprise": 0 }
    
    for emotion, intensity in intensity_emotion.items():
        if intensity in ["baixo", "moderado", "alto"] and emotion != "Neutral":
            emotions[emotion] = 1
        
        
    entry = {
        "id": example_id,
        "text": text,
        **emotions
    }
    
    data[str(n)] = entry
    json_controller.save_data(data)
    
    #time.sleep(1)
    i += 1

Generating example 2909/3000   -  EMOTION: {'Disgust': 'baixo', 'Sadness': 'baixo', 'Surprise': 'baixo'}    -   CATEGORY: Pequeno
Generating example 2910/3000   -  EMOTION: {'Disgust': 'baixo', 'Sadness': 'moderado'}    -   CATEGORY: Pequeno
Generating example 2911/3000   -  EMOTION: {'Anger': 'baixo', 'Fear': 'baixo'}    -   CATEGORY: Pequeno
Generating example 2912/3000   -  EMOTION: {'Anger': 'moderado', 'Fear': 'baixo', 'Sadness': 'moderado'}    -   CATEGORY: Grande
Generating example 2913/3000   -  EMOTION: {'Disgust': 'baixo', 'Surprise': 'alto'}    -   CATEGORY: Grande
Generating example 2914/3000   -  EMOTION: {'Joy': 'baixo', 'Surprise': 'baixo'}    -   CATEGORY: Pequeno
Generating example 2915/3000   -  EMOTION: {'Fear': 'moderado'}    -   CATEGORY: Grande
Generating example 2916/3000   -  EMOTION: {'Fear': 'baixo', 'Joy': 'moderado', 'Sadness': 'moderado'}    -   CATEGORY: Médio
Generating example 2917/3000   -  EMOTION: {'Fear': 'baixo'}    -   CATEGORY: Médio
Generating ex

In [ ]:
#todo: criar loop para geração dos dados
#todo estruturar prompt
#todo lidar com json de saída
#todo salvar dados e subir no hf

#### Transformando formato dos dados

In [18]:
dados = json_controller.load_data()

In [19]:
generated = pd.DataFrame.from_dict(dados, orient='index')
generated.reset_index(inplace=True)
generated.rename(columns={'index': 'ID'}, inplace=True)
generated.columns = ["ID","indice" ,"text", "anger", "disgust", "fear", "joy", "sadness", "surprise"]

generated

,ID,indice,text,Anger,Disgust,Fear,Joy,Sadness,Surprise
0,0,synthetic_0,Hoje foi um dia cheio de altos e baixos. Comec...,0,0,0,1,1,1
1,1,synthetic_1,"Puts, vi que a galera tá falando de novo daque...",1,0,0,0,1,1
2,2,synthetic_2,"Hoje resolvi dar uma volta no parque, só pra p...",0,0,1,1,0,0
3,3,synthetic_3,"Tô pensando aqui, é estranho como a vida é che...",0,0,1,1,1,0
4,4,synthetic_4,Não consigo entender como algumas pessoas aind...,1,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...
2995,2995,synthetic_2995,"Putz, decidi fazer um lanche, mas quando abri ...",0,1,0,0,0,0
2996,2996,synthetic_2996,"Olha só, cansei desse papo de que tudo tá ótim...",1,1,0,0,1,0
2997,2997,synthetic_2997,"Sinceramente, fiquei chocado com a notícia que...",0,1,0,0,1,1
2998,2998,synthetic_2998,"Hoje foi um dia meio complicado, sabe? Acordei...",0,0,0,1,1,0


In [26]:
'''emotions = ['Anger', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise']
total_per_emotion = 500

def get_emotion_by_index(index):
    emotion_index = index // total_per_emotion
    return emotions[emotion_index % len(emotions)]

for index, row in generated.iterrows():
    if all(row[emotion] == 0 for emotion in emotions):
        emotion_to_update = get_emotion_by_index(index)
        generated.at[index, emotion_to_update] = 1
        
generated = generated.drop(columns=['ID'])
generated = generated.rename(columns={'indice': 'id'})'''

In [20]:
generated

,ID,indice,text,Anger,Disgust,Fear,Joy,Sadness,Surprise
0,0,synthetic_0,Hoje foi um dia cheio de altos e baixos. Comec...,0,0,0,1,1,1
1,1,synthetic_1,"Puts, vi que a galera tá falando de novo daque...",1,0,0,0,1,1
2,2,synthetic_2,"Hoje resolvi dar uma volta no parque, só pra p...",0,0,1,1,0,0
3,3,synthetic_3,"Tô pensando aqui, é estranho como a vida é che...",0,0,1,1,1,0
4,4,synthetic_4,Não consigo entender como algumas pessoas aind...,1,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...
2995,2995,synthetic_2995,"Putz, decidi fazer um lanche, mas quando abri ...",0,1,0,0,0,0
2996,2996,synthetic_2996,"Olha só, cansei desse papo de que tudo tá ótim...",1,1,0,0,1,0
2997,2997,synthetic_2997,"Sinceramente, fiquei chocado com a notícia que...",0,1,0,0,1,1
2998,2998,synthetic_2998,"Hoje foi um dia meio complicado, sabe? Acordei...",0,0,0,1,1,0


In [21]:
generated.to_csv('data/balancing_sample_1.csv', index=False)

#### Subindo para o hugging face